# Import Required Libraries

In [1]:
import re
from rapidfuzz import fuzz
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
import jdatetime

### Load Dataset

In [2]:
df = pd.read_csv('reviews.csv')

### Basic Cleaning

In [3]:
"""
During EDA, we determined that nearly all missing values were due to: 
survey images, professor suggestion polls, or various single messages 
(like pinned posts, guides, or initial channel messages).
We may revisit EDA-related code later to extract any potentially useful information from these if possible.
For now, we removing them as we did during the EDA phase.
"""

# Just 2 Special Cases
df.loc[df['message_id'] == 816, ['department', 'comment_text', 'parse_error']] = [
    'معارف', 
    'خییلللللی استاد محشرین از دسشون ندین ۴ نمره اضافی دارن برای نهاد که جواباش پیدا میشه😁 امتحاناشونم میانترم کپی ترم پیش و پایانترم از یه فایل ۲۰ صفحه ای بود',
    None
]

df.loc[df['message_id'] == 252, 'comment_text'] = np.nan

df = df.dropna(subset=['term'])

In [4]:
print("Total reviews:", len(df))
print("Unique professors:", df["professor_name_raw"].nunique())
print("Departments:", df["department"].nunique())

Total reviews: 4276
Unique professors: 1296
Departments: 18


In [5]:
df['comment_text'].isna().sum()

16

### Term Normalization

In [6]:
temp_grouped = {
 ('قبل از سال 94',): 'before_94',

 ('بهمن 94',): 'second_1394',

 ('تابستان 95',): 'summer_1395',
 
 ('مهر 96',): 'first_1396',
 ('تابستان 96',): 'summer_1396',
 
 ('مهر 97',): 'first_1397',
 ('بهمن 97',): 'second_1397',
 
 ('مهر 98',): 'first_1398',
 ('بهمن 98',): 'second_1398',
 
 ('مهر 99',): 'first_1399',
 ('بهمن 99', 'بهمن ۹۹'): 'second_1399',
 ('تابستان 99',): 'summer_1399',
 
 ('مهر 00', 'مهر 1400', 'مهر ۰۰', 'مهر ..', 'مهر ١٤٠٠', 'مهر ۱۴۰۰', 'مهر۱۴۰۰', 'پاییز ۱۴۰۰'): 'first_1400',
 ('4002', 'بهمن 1400', 'بهمن ۱۴۰۰'): 'second_1400',
 ('تابستان 1400', 'تابستان۱۴۰۰'): 'summer_1400',
 ('بهمن 99 ،مهر 1400', 'مهر و بهمن 1400', 'مهر و بهمن ۱۴۰۰', 'هم ترم مهر ۱۴۰۰ و ترم بهمن ۱۴۰۰'): 'both_1400',
 
 ('14011', 'ترم های 4001 - 4002 - 4011', 'مهر 1401', 'مهر ۴۰۱', 'پاییز ۱۴۰۱'): 'first_1401',
 ('بهار 1402', 'بهار ۱۴۰۲', 'بهار ۴۰۲', 'بهمن 1401', 'بهمن ۴۰۱', 'فرودین ۱۴۰۲'): 'second_1401',
 ('تابستان 1401',): 'summer_1401',
 ('مهر 1401 و بهمن 1402',): 'both_1401',

 ('1402 پاییز', 'مهر 1402', 'مهر 402', 'مهر ۱۴۰۲', 'مهر ۴۰۲', 'مهر۱۴۰۲', 'مهر۴۰۲', 'مهر 1402 - بهمن 1402', 'مهر ۱۴۰۲-بهمن ۱۴۰۲', 'بهمن۱۴۰۱_مهر۱۴۰۲', 'پاییز  ۱۴۰۲', 'پاییز 1402', 'پاییز 402', 'پاییز ۱۴۰۲', '۱۴۰۲ مهر', '۱۴۰۲-۱'): 'first_1402',
 ('4022', 'اسفند 1402', 'اسفند ۱۴۰۲', 'بهار 1403', 'بهار ۱۴۰۳', 'بهار ۴۰۳', 'بهار 403', 'بهتر ۴۰۳', 'بهمن ۱۰۴۲', 'بهمن ۴۰۲ ۴۰۳', 'بهمن۴۰۲-۳', 'بهمن 1402', '1402 بهمن', '1402بهمن', 'بهمن  1402', 'بهمن 1402-1403', 'بهمن 402', 'بهمن ۱۴۰۲', 'بهمن ۴۰۲', 'بهمن۱۴۰۲', 'ترم دو ۱۴۰۲_۱۴۰۳', 'زمستان ۱۴۰۲', 'زمستان ۴۰۲', 'زمستان 402', 'خرداد ۴۰۳', 'زمستان ۱۴۰۲ و بهار ۱۴۰۳', 'زمستان ۴۰۳', '۱۴۰۲٫۱۴۰۳', '۱۴۰۲_۱۴۰۳', '۴۰۲.۲'): 'second_1402',
 ('تابستان 1402',): 'summer_1402',
 ('1402', '۱۴۰۲', 'اول و دوم ۱۴۰۲', 'مهر 1402 و بهمن 1402', 'مهر و بهمن ۴۰۲', '۱۴۰۲ ۱۴۰۳'): 'both_1402',
 
 ('1403 مهر', '403', '4031', 'مهر 1403', 'مهر 403', 'مهر ۱۴٠۳', 'مهر ۱۴۰۳', 'مهر ١۴٠٣', 'مهر ۴۰۳', 'مهر1403', 'مهر403', 'مهر۱۴۰۳', 'مهر۴۰۳', 'مهر۴۰۳- بهمن۴۰۳', 'نیمه اول ۱۴۰۳', 'پائیز 1403', 'پاییز 1403', 'پاییز 403', 'پاییز ۱۴۰۳', 'پاییز ۴۰۳', 'پاییز۴۰۳'): 'first_1403',
 ('1403-2', '4032', '403_2', 'بهار ۱۴۰۴', 'بهار ۴۰۴','بهمن 1403', 'بهمن 403', 'بهمن ١٤٠٣', 'بهمن ۱۴٠۳', 'بهمن ۱۴۰۳', 'بهمن ۴۰۳', 'بهمن۱۴۰۳', 'بهمن ۴۰۲-۳', 'بهمن۴۰۳', 'ترم زمستان 1403', '۴۰۳-۲', '۴۰۳۲'): 'second_1403',
 ('تابستان 1403', 'تابستان 403', 'تابستان ۱۴۰۳', 'تابستان ۴۰۳'): 'summer_1403',
 ('۱۴۰۳', '۴۰۳', 'آبان و بهمن ۱۴۰۳', 'بهمن ۱۴۰۳ و مهر ۱۴۰۳', 'مهر و بهمن 1403', 'مهر و بهمن ۴۰۳', 'مهر ۱۴۰۳ و بهمن ۴۰۳-۴۰۴', 'مهر۱۴۰۳و بهمن ۱۴۰۳'): 'both_1403',

 ('مهر 1404', '۴۰۴'): 'first_1404',
 ('بهمن 1404', 'بهمن ۱۴۰۴', 'بهمن ۴۰۴'): 'second_1404',
 ('تابستان 1404', 'تابستان ۴٠۴'): 'summer_1404'
}

flat_mapping = {
    key: value
    for keys, value in temp_grouped.items()
    for key in keys
}

In [7]:
df['term'] = (
    df['term']
      .astype(str)
      .str.strip()
      .str.replace(r'([آ-یA-Za-z]+)([0-9]+)', r'\1 \2', regex=True)
      .map(flat_mapping)
)

In [8]:

prefixes = ['دکتر', 'دکتری', 'دكتر', 'مهندس', 'مهندسی', 'آقای', 'آقا', 'خانم', 'خانوم', 'استاد', 'استادی', 'پروفسور', 'دستیار', 'معاون', 'سرکار', 'سرکار خانم', 'جناب', 'جناب آقای', 'سرپرست', 'مدیر', 'رییس', 'رئیس']

pc = {p: 0 for p in prefixes}
for n in df['professor_name_raw']:
    s = str(n).strip()
    for p in prefixes:
        if s.startswith(p):
            pc[p] += 1
            break

for p, c in sorted(pc.items(), key=lambda x: -x[1]):
    if c: print(f"{p}: {c}")

دکتر: 153
استاد: 88
خانم: 2
پروفسور: 2
مهندس: 1
آقای: 1
آقا: 1


### Professor Names Normalization